## 빅데이터 실습
### 와인품질 등급 분석/예측
#### 데이터 수집
##### UC 어바인 머신러닝 연구소 데이터 다운로드
- ucimlrepo(종속성 dotdict 모듈 같이 설치) [데이터](https://archive.ics.uci.edu/dataset/186/wine+quality)
##### statsmodes 라이브러리 분석 함수
- OLS : Ordinary Least Square 2승 모델로 분석을 하는 함수
- GLM : Generalized Linear Model 일반 선형모델로 분석 함수
- GLS : Generalized Least Square 2승 일반화된 최소 승수 모델(두가지로 표본집단을 나눠서)
- WLS : Weighted Least Squre 가중치를 부가한 OLS 모델
- fit() : 훈련결과를 내는 함수
- predict() : 결과를 예측하는 함수

In [ ]:
## 필수 라이브러리/모듈 사용등록
import pandas as pd
from ucimlrepo import fetch_ucirepo  # 인터넷으로 통계 데이터를 다운로드함 함수
from scipy import stats # 과학수치모듈 내의 기술통계 객체 사용
from statsmodels.formula.api import ols, glm, wls, gls # 분석을 위한 결과를 도출해주는 함수들

In [ ]:
## 와인 데이터가져오기
wineQuality = fetch_ucirepo(id=186)

In [ ]:
dfWineTotal = wineQuality.data.original

In [ ]:

dfWineTotal.describe()

##### 회귀분석

In [ ]:
## 회귀분석 변수로 회귀수식 작성
RFormula = 'quality ~ fixed_acidity + volatile_acidity + citric_acid + residual_sugar + chlorides+ free_sulfur_dioxide + total_sulfur_dioxide + density + pH + sulphates + alcohol'

In [ ]:
## 회귀분석
regrResult = ols(RFormula, data=dfWineTotal).fit()
regrResult.summary()

In [ ]:
regrResult2 = glm(RFormula, data=dfWineTotal).fit()
regrResult2.summary()

##### 도출된 회귀분석 결과로 새로운 와인샘플 품질등급 예측

In [ ]:
## 와인 품질등급 예측용 샘플데이터
dfWineTotal.columns.difference(['quality', 'color'])

In [ ]:

# 품질예측할것이기때문에 quality 불필요, 와인종류도 의미없음
sample1 = dfWineTotal[dfWineTotal.columns.difference(['quality', 'color'])]

In [ ]:
sample1 = sample1[0:5]
sample1

In [ ]:
dfWineTotal.head()

In [ ]:
# OLS 모델 예측시작
regrResult.predict(sample1)

In [ ]:
# GLM 모델 예측
regrResult2.predict(sample1)

##### 결론 : 실제 데이터와 선형회귀 모델에서 예측한 결과가 거의 동일

In [ ]:
# 현재 실제 데이터 중 와인 등급이 9 것
condition = dfWineTotal['quality'] == 9
dfWineTotal[condition]

In [ ]:
# 9등급 와인만 예측
dfWineTop = dfWineTotal[condition]

In [ ]:
sample3 = dfWineTop[dfWineTop.columns.difference(['quality', 'color'])]

In [ ]:
sample3

In [ ]:
dfWineTotal[condition].describe()

In [ ]:
## 더미데이터 생성, 세번째 값은 와인품질 9등 평균으로 만든 값
data = {'fixed_acidity': [8.5, 8.1, 7.5, 7.1] ,'volatile_acidity': [0.8, 0.5, 0.3, 0.26] , 'citric_acid': [0.3, 0.4, 0.4, 0.49], 'residual_sugar': [6.1, 5.8, 5.0, 2.2], 
        'chlorides': [0.055, 0.04, 0.027, 0.032], 'free_sulfur_dioxide': [30.0, 31.0, 33.4, 31.0], 'total_sulfur_dioxide': [98.0, 99, 116.0, 113.0], 'density': [0.996, 0.91, 0.991, 0.99030], 
        'pH': [3.15, 3.01, 3.3, 3.37], 'sulphates': [0.4, 0.35, 0.47, 0.42], 'alcohol': [9.0, 0.88, 12.2, 12.9]}

In [ ]:
## 컬럼 순서를 바꿈. sample1으로 예측한 데이터와 컴럼순서를 일치시키기 위해서 columns=sample1.columns 필요없음, 결과에 아무런 영향을 미치지 않음
sample2 = pd.DataFrame(data, columns=sample1.columns)
sample2

In [ ]:
# 예측 시작
regrResult.predict(sample2)

In [ ]:
regrResult2.predict(sample2)

In [ ]:
# 9등급 와인만 재예측
regrResult.predict(sample3)

##### 결론 시각화

In [ ]:
## 시각화 라이브러리 사용등록
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm # 선형회귀 시각화를 위한 객체

In [ ]:

# 한글 깨짐 문제 해결
from matplotlib import rcParams, font_manager, rc

font_path = 'C:/Windows/Fonts/NanumGothicCoding.ttf' # 나눔고딕코딩 사용, 나눔고딕에서 오류발생(!)
font = font_manager.FontProperties(fname=font_path).get_name() # 실제 설치된 폰트 이름조회
rc('font', family=font) # 한글깨짐현상 해결!!
rcParams['axes.unicode_minus'] = False # 한글 사용시 마이너스 표시 깨짐 해결!

In [ ]:
sns.set_style('darkgrid')

In [ ]:
# 레드와인품질 만
condition = dfWineTotal['color'] == 'red'
redWineQuality = dfWineTotal[condition]['quality']

In [ ]:
# 화이트와인품질 만
condition = dfWineTotal['color'] == 'white'
whiteWineQuality = dfWineTotal[condition]['quality']

In [ ]:
redWineQuality.info()

In [ ]:
whiteWineQuality.info()

In [ ]:
sns.__version__

In [ ]:
# distplot() seaborn v0.14.0 에서 제거될 함수, 대신 histplot() 사용할 것
sns.histplot(redWineQuality, kde=True, color='red', label='레드와인')
sns.histplot(whiteWineQuality, kde=True, color='white', label='화이트와인')

plt.title('와인종류별 품질')
plt.legend()
plt.show()

In [ ]:
dfWineTotal.tail()

In [ ]:
others = list(set(dfWineTotal.columns).difference(set['quality', 'fixed_acidity']))

In [ ]:
# fixed_acidity(고정산)에대한 quality(품질) 의 상관관계를 선형회귀분석으로 시각화
sm.graphics.plot_partregress(exog_i='fixed_acidity', endog='quality', exog_others=others, data=dfWineTotal, obs_labels=False)

plt.show()

In [ ]:
# 선형회귀 결과 시각화
fig = plt.figure(figsize=(10, 14))
sm.graphics.plot_partregress_grid(regrResult, fig=fig)
plt.show()

In [ ]:
dfWineTotal[['alcohol', 'quality']]